In [1]:
# Importing required packages 
import pandas as pd
import numpy as np

In [2]:
# Importing glossaries for replacing numeric labels in a dataframe with their string counterparts 
import dictionaries as pums_dict

In [3]:
# create a list with preferred column names 
pref_names = ['Data Year', 'State', 'Age', 'Citizenship', 'Marital Status', 'Military Status', 'Work availability', 'Laid Off', 'Looking for Work', 'School Enrollment','Grade Level Attending', 'Educational Attainment', 'Sex', 'First Degree', 'Second Degree', 'Occupation', 'Children Age', 'Total Income']

# create a list with columns to be selected from the dataset
select_cols = ['ST', 'AGEP', 'CIT', 'MAR', 'MIL', 'NWAV',  'NWLA', 'NWLK', 'SCH', 'SCHG', 'SCHL', 'SEX', 'FOD1P', 'FOD2P', 'OCCP', 'PAOC', 'PINCP']


In [4]:
# open the CSV file with columns of choice 
wa_data_20 = pd.read_csv('wa_2020_pums.csv', usecols = select_cols)
id_data_20 = pd.read_csv('id_2020_pums.csv', usecols = select_cols)
or_data_20 = pd.read_csv('or_2020_pums.csv', usecols = select_cols)

In [5]:
# concatenate into a single dataframe 
pums_2020 = pd.concat([wa_data_20, id_data_20, or_data_20], axis = 0)

In [6]:
# add new column at the beginning of the dataframe to differentiate between 2019 and 2020 data 
pums_2020.insert(0, 'PUMS_Year', 2020)

In [7]:
# Multiplying total income by an adjustment factor set for 2020; rounded to the whole number since we later will be changing to integer type 
pums_2020['PINCP'] = round(1.006149 * pums_2020['PINCP'], 0)

In [8]:
# open the CSV file with columns of choice 
wa_data_19 = pd.read_csv('wa_2019_pums.csv', usecols = select_cols)
id_data_19 = pd.read_csv('id_2019_pums.csv', usecols = select_cols)
or_data_19 = pd.read_csv('or_2019_pums.csv', usecols = select_cols)

In [9]:
# concatenate into a single dataframe 
pums_2019 = pd.concat([wa_data_19, id_data_19, or_data_19], axis = 0)

In [10]:
# add new column at the beginning of the dataframe to differentiate between 2019 and 2020 data 
pums_2019.insert(0, 'PUMS_Year', 2019)

In [11]:
# Multiplying total income by an adjustment factor set for 2020; rounded to the whole number since we later will be changing to integer type 
pums_2019['PINCP'] = round(1.010145 * pums_2019['PINCP'], 0)

In [12]:
# combine 2019 and 2020 PUMS data into a single dataset
pums_data = pd.concat([pums_2019, pums_2020], axis = 0)

In [13]:
# set new headers 
pums_data.set_axis(pref_names, axis = 1, inplace=True)

In [14]:
# switch NA values to 0 (need int instead of float for the dictionaries to work, cannot apply astype() without achieving uniformity of data types in all columns)
pums_data = pums_data.fillna(0)

In [15]:
# change values in all columns from float to int
pums_data = pums_data.astype(int)

In [16]:
# substitute numeric values in columns with labels from the imported dictionaries 
pums_data = pums_data.replace({'State': pums_dict.states, 'Citizenship': pums_dict.citizenship, 'Marital Status': pums_dict.marital_status, 'Military Status': pums_dict.military_service, 
            'Work availability': pums_dict.work_available, 'Laid Off': pums_dict.laid_off, 'Looking for Work': pums_dict.work_available, 'School Enrollment': pums_dict.enroll_status,
            'Grade Level Attending': pums_dict.grade_level, 'Educational Attainment': pums_dict.ed_attainment, 'Sex': pums_dict.sex, 'First Degree': pums_dict.education_field, 'Second Degree': pums_dict.education_field,
            'Children': pums_dict.children, 'Occupation': pums_dict.occupation, 'Children Age': pums_dict.children_age})

In [17]:
# reindex the columns to improve logic in data representation
new_order = ['Data Year', 'State', 'Age', 'Sex', 'Citizenship','School Enrollment','Grade Level Attending', 'Educational Attainment', 'First Degree', 'Second Degree', 'Military Status', 'Work availability', 'Laid Off', 'Looking for Work', 'Occupation', 'Marital Status', 'Children Age', 'Total Income']
pums_data = pums_data.reindex(columns = new_order)


In [18]:
# change first index from 0 to 1 
pums_data.index = np.arange(1, len(pums_data) + 1)

In [28]:
# check percentage for missing values in each column and create a dataframe to visualize this info
na_percentage = pums_data.replace(0, np.NaN).isnull().sum() * 100 / len(pums_data)
na_report = pd.DataFrame({'na percentage': na_percentage})
na_report.index.name = 'Column'

# sort values in a descending order and print top 5 with head()
na_report.sort_values('na percentage', ascending = False, inplace = True)
na_report.head()

,na percentage
Column,
Second Degree,96.517224
Grade Level Attending,78.144789
First Degree,72.813371
Children Age,60.173268
Occupation,39.944276


In [19]:
# create a DataFrame which contains all records for current students whose age is 24 and up (segregate by year)
nontrad_stud_2019 = pums_data.loc[(pums_data['Age'] >= 24) & (pums_data['Grade Level Attending'].str.contains('graduate', case = False)) & (pums_data['Data Year'] == 2019)]
nontrad_stud_2020 = pums_data.loc[(pums_data['Age'] >= 24) & (pums_data['Grade Level Attending'].str.contains('graduate', case = False)) & (pums_data['Data Year'] == 2020)]

In [20]:
# reindex nontrad_students 
nontrad_stud_2019.index = np.arange(1, len(nontrad_stud_2019) + 1)
nontrad_stud_2020.index = np.arange(1, len(nontrad_stud_2020) + 1)

In [25]:
# save resulting datasets to CSV file (for further analysis in PowerBI)
nontrad_stud_2019.to_csv("nontrad_stud_2019.csv")
nontrad_stud_2020.to_csv("nontrad_stud_2020.csv")

In [86]:
# Check percentage of nontrad students reported as laid off or looking for work in 2019 vs 2020 

def work_search(data):
    work_search = data.loc[(data['Laid Off'] == 'Yes') | (data['Looking for Work'] == 'Yes')]
    try:
        percentage_work_search = round(len(work_search) * 100 / len(data), 2)
        return percentage_work_search
    except ZeroDivisionError:
        print("No records")

print("Percentage of nontrad students reported as laid off or looking for work:\n")
print(f"\t- 2019 data: {work_search(nontrad_stud_2019)}%")
print(f"\t- 2020 data: {work_search(nontrad_stud_2020)}%")


Percentage of nontrad students reported as laid off or looking for work:

	- 2019 data: 5.93%
	- 2020 data: 7.21%


In [87]:
# Check percentage of past or current military listed as students in 2019 vs 2020

def roti_stud(data):
    roti_stud = data.loc[(data['Military Status'].str.contains('active duty', case = False))]
    try:
        percentage_roti = round(len(roti_stud) * 100 / len(data), 2)
        return percentage_roti
    except ZeroDivisionError:
        print("No records")

print("Percentage of past or current military listed as students:\n")
print(f"\t- 2019 data: {roti_stud(nontrad_stud_2019)}%")
print(f"\t- 2020 data: {roti_stud(nontrad_stud_2020)}%")


Percentage of past or current military listed as students:

	- 2019 data: 12.01%
	- 2020 data: 10.31%


In [80]:
# Let's check median income for 2019, based on income groups, as grouped by state and gender 
# First we create a subframe with selected columns from nontrad_stud_2019 dataframe
income_comparison_2019 = pd.DataFrame().assign(State = nontrad_stud_2019['State'], Sex = nontrad_stud_2019['Sex'], Income = nontrad_stud_2019['Total Income'])
# Create a new column "Income Category" which buckets records based on income margins
income_comparison_2019 ['Income Category'] = pd.cut(nontrad_stud_2019['Total Income'], bins = [0, 52200, 156600, 1000000], labels = ['low', 'middle', 'upper'])
# Getting rid of NA values since we don't need them 
income_comparison_2019 = income_comparison_2019.dropna()
# Calculate median income for each income category based on sex and state 
income_comparison_2019.groupby(['State','Sex', 'Income Category']).median()

# Note that median is preferred since it doesn't take into account the outliers!


Income
State Sex    Income Category          
ID    Female low               19193.0
             middle            60609.0
             upper            316175.5
      Male   low               20405.0
             middle            74296.5
             upper            313953.0
OR    Female low               20203.0
             middle            68690.0
             upper            181826.0
      Male   low               21213.0
             middle            78791.0
             upper            172735.0
WA    Female low               21819.0
             middle            70710.0
             upper            222232.0
      Male   low               24243.0
             middle            77801.5
             upper            207079.5

In [81]:
# Performing the same steps for 2020 data
income_comparison_2020 = pd.DataFrame().assign(State = nontrad_stud_2020['State'], Sex = nontrad_stud_2020['Sex'], Income = nontrad_stud_2020['Total Income'])
income_comparison_2020 ['Income Category'] = pd.cut(nontrad_stud_2020['Total Income'], bins = [0, 52200, 156600, 1000000], labels = ['low', 'middle', 'upper'])
income_comparison_2020 = income_comparison_2020.dropna()
income_comparison_2020.groupby(['State','Sex', 'Income Category']).median()

Income
State Sex    Income Category          
ID    Female low               20324.0
             middle            62894.5
             upper                 NaN
      Male   low               22135.0
             middle            82504.5
             upper            202035.0
OR    Female low               20123.0
             middle            70430.0
             upper            229402.0
      Male   low               19620.0
             middle            74706.5
             upper            187646.5
WA    Female low               21733.0
             middle            72443.0
             upper            201230.0
      Male   low               22135.0
             middle            80291.0
             upper            196702.5

In [124]:
# Checking reported number of people who were laid-off or actively seeking employment in 2019, as grouped by sex and state
# Selecting required columns
layoffs_2019 = pd.DataFrame().assign(State = nontrad_stud_2019['State'], Sex = nontrad_stud_2019['Sex'], Layoff = nontrad_stud_2019['Laid Off'], WorkAvail = nontrad_stud_2019['Looking for Work'])
# Filtering out records where either being laid off or looking for work is reported as "yes"
layoffs_2019 =  layoffs_2019.loc[(layoffs_2019['Layoff'] == 'Yes') | (layoffs_2019['WorkAvail'] == 'Yes')]
# Combining two columns for easier calculation of the total number of records for both
layoffs_2019['Layoff'] = layoffs_2019['Layoff'] + layoffs_2019['WorkAvail']
# replacing all values with "Yes"
layoffs_2019 = layoffs_2019.assign(Layoff = 'Yes')
# removing 'WorkAvail' column since it is no longer needed 
layoffs_2019 = layoffs_2019.drop(['WorkAvail'], axis = 1)
# displaying the grouped output
layoffs_2019.groupby(['State', 'Sex']).count()


Layoff
State Sex           
ID    Female       9
      Male        12
OR    Female      30
      Male        32
WA    Female      72
      Male        72

In [125]:
# Performing the same set of operations for 2020 report 
layoffs_2020 = pd.DataFrame().assign(State = nontrad_stud_2020['State'], Sex = nontrad_stud_2020['Sex'], Layoff = nontrad_stud_2020['Laid Off'], WorkAvail = nontrad_stud_2020['Looking for Work'])
layoffs_2020 =  layoffs_2020.loc[(layoffs_2020['Layoff'] == 'Yes') | (layoffs_2020['WorkAvail'] == 'Yes')]
layoffs_2020['Layoff'] = layoffs_2020['Layoff'] + layoffs_2020['WorkAvail']
layoffs_2020 = layoffs_2020.assign(Layoff = 'Yes')
layoffs_2020 = layoffs_2020.drop(['WorkAvail'], axis = 1)
layoffs_2020.groupby(['State', 'Sex']).count()

Layoff
State Sex           
ID    Female      12
      Male        13
OR    Female      33
      Male        28
WA    Female      71
      Male        64